In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import data_preperation as prep
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

#example of how to call preparation.py
import data_process as proc
import data_preperation as prep
from utils_scenario import *


In [2]:
#example of how to use data_process.py

#call data_preperation.py to get data 
measurement=prep.prepare_measurement()
forecast = prep.prepare_forecast()
#keep useful columns 
measurement= measurement[['speed', 'cos_wind_dir', 'sin_wind_dir', 'temp', 'radiation', 'precip', 'season', 'am']]

#call data_process.py
x_df, y_df, x, y_speed = proc.prepare_x_y(measurement, forecast, 24, 24, 'speed')
x_df, y_df, x, y_cos = proc.prepare_x_y(measurement, forecast, 24, 24, 'cos_wind_dir')
x_df, y_df, x, y_sin = proc.prepare_x_y(measurement, forecast, 24, 24, 'sin_wind_dir')

read csv semester csv files from 2015s2 to 2020s1
smooth wind direction
generate seasonality categorical feature
generate am/pm categorical feature
reading forecast data
smooth wind direction


In [3]:
y_scenarios = get_all_scenarios(y_speed, y_cos, y_sin, b_scenarios = True)
y_dangerous = get_all_dangerous_scenarios(y_speed, y_cos, y_sin)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(x, y_scenarios, test_size=0.2, shuffle = False)

In [37]:
xg_speed = XGBRegressor(max_depth = 5)
xg_speed.fit(X_train, y_train)
y_hat = xg_speed.predict(X_test)

[14:35:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: 

In [38]:
xg_dangerous_scenario = XGBClassifier(max_depth = 3)
xg_dangerous_scenario.fit(X_train, y_train)
y_hat = xg_dangerous_scenario.predict(X_test)

In [32]:
100-203/7994*100

97.46059544658493

# Metrics

In [39]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           1       0.87      0.97      0.92      6671
           2       0.49      0.34      0.40      1015
           3       0.71      0.02      0.03       308
           4       0.21      0.06      0.09       126
           5       0.00      0.00      0.00        45
           6       0.07      0.03      0.04        32

   micro avg       0.83      0.83      0.83      8197
   macro avg       0.39      0.23      0.25      8197
weighted avg       0.80      0.83      0.80      8197



In [40]:
print("Accuracy for classification based prediction is:", accuracy_score(y_test, y_hat))

Accuracy for classification based prediction is: 0.8315237281932414
